In [6]:
from dcicutils import ff_utils
from functions.notebook_functions import *

# get key from file
my_key = get_key('koray_data')
# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'data')

In [7]:
q = '/search/?experimentset_type=replicate&type=ExperimentSetReplicate&experiments_in_set.experiment_type=CUT%26RUN&experiments_in_set.experiment_type=ChIP-seq&experiments_in_set.experiment_type=NAD-seq&status=pre-release&status=released&status=released%20to%20project'
gen = ff_utils.search_metadata(q, key=my_key)
print(len(gen))

102


In [21]:
l = ['4DNESO2RRVT8', '4DNES6VO5VSG', '4DNESKIB4QKT', '4DNESVCBVRHM', '4DNES7KIE2LD', '4DNES49QC878', '4DNESIO87EN4', '4DNESPY1NI26', '4DNESNRB3TT6', '4DNES9GHAPEX', '4DNESWBS1FVN', '4DNESOMKLZSI', '4DNESG1WF34H', '4DNESZ8BKGUV', '4DNESEZVPO8G', '4DNESRIK1N34', '4DNESK46EYO5', '4DNESHX5CM31', '4DNESMX5EBFU', '4DNESRHWL6VD']

for f in gen:

    #f = ff_utils.get_metadata(le, key=my_key)
    desc = f['description']
    datas = f.get('dataset_label',"")
    condition = f.get('condition',"")
    replicate_exps = f['replicate_exps']
    replicate_exps = sorted(replicate_exps, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])
    r_no = len(replicate_exps)
    for n, i in enumerate(replicate_exps):
        paired = ""
        b = i['bio_rep_no']
        t = i['tec_rep_no']
        expacc = i['replicate_exp']['@id'].split('/')[2]
        exp_resp = ff_utils.get_metadata(expacc, key=my_key)
        exp_type = exp_resp['experiment_type']
        
        biosample = exp_resp['biosample']
        organism = list(set([bs['individual']['organism']['name'] for bs in biosample['biosource']]))[0]

        control = ''
        exp_relation = exp_resp.get('experiment_relation')
        if exp_relation:
            rel_type = [i['relationship_type'] for i in exp_relation]
            if 'control for' in rel_type:
                control = "control experiment"
                break
            elif 'controlled by' in rel_type:
                controls = [i['experiment'] for i in exp_relation if i['relationship_type'] == 'controlled by']
        
                if len(controls) != 1:
                    control = 'MULTI ' + ", ".join([i['@id'].split('/')[2] for i in controls])
                else:
                    control = controls[0]['@id'].split('/')[2]
        else:
            control = 'no info'
        size = str(exp_resp.get('average_fragment_size', ""))
        size_r = exp_resp.get('fragment_size_range', "")
        
        
        
        # get exp files
        exp_files = exp_resp['files']
        for a_file in exp_files:
            file_resp = ff_utils.get_metadata(a_file['uuid'], key = my_key)           
            # get pair end no
            pair_end = file_resp.get('paired_end')

            if not pair_end:
                paired = 'no'  

            if pair_end == '2':
                paired = 'yes'

            if pair_end == '1':
 
                relations = file_resp.get('related_files')

                
                if not relations:
                    paired = 'no'
                    print('d')
                else:
                    for relation in relations:
                        if relation['relationship_type'] == 'paired with':
                            paired = 'yes'
            break
 
        target = exp_resp.get('targeted_factor')
        if target:
            target_n = target['display_title']
        else:
            target_n = 'None'
        
        print(";".join([f['accession'],exp_type, organism,target_n,desc, datas, condition, expacc, str(b), str(t), control, paired, size, size_r]))
            

4DNESO2RRVT8;ChIP-seq;mouse;Protein:H3K27me3;ChIP-seq of  CTCF-AID washed off mouse embryonic stem cells after being treated with auxin targetting H3K27me3 gene;;;4DNEXRZ99IR1;1;1;4DNEX7ICLCEK;no;;
4DNESO2RRVT8;ChIP-seq;mouse;Protein:H3K27me3;ChIP-seq of  CTCF-AID washed off mouse embryonic stem cells after being treated with auxin targetting H3K27me3 gene;;;4DNEXKASK5O3;2;1;4DNEXZTMNGRQ;no;;
4DNES6VO5VSG;ChIP-seq;mouse;Protein:H3K27me3;ChIP-seq of CTCF-AID mouse embryonic stem cells treated with Auxin for 4 days targetting H3K27me3 gene;;;4DNEXX785YMJ;1;1;4DNEX7ICLCEK;no;;
4DNES6VO5VSG;ChIP-seq;mouse;Protein:H3K27me3;ChIP-seq of CTCF-AID mouse embryonic stem cells treated with Auxin for 4 days targetting H3K27me3 gene;;;4DNEXT9LL8UQ;2;1;4DNEXZTMNGRQ;no;;
4DNESKIB4QKT;ChIP-seq;mouse;Protein:H3K27me3;ChIP-seq of CTCF-AID mouse embryonic stem cells treated with Auxin for 2 days targetting H3K27me3 gene;;;4DNEXN8YEAD3;1;1;4DNEX7ICLCEK;no;;
4DNESKIB4QKT;ChIP-seq;mouse;Protein:H3K27me3;ChIP

4DNESPVOADIG;CUT&RUN;human;Protein:H2A.Z (P0C0S5);Replicates CUT&RUN experiments on DE cells using H2A.Z (Active Motif 39113) with a digestion time of 3min;;;4DNEX5D6Q6C2;2;1;4DNEXZPAE6HZ;yes;;
4DNESN18Y5S3;CUT&RUN;human;Protein:CTCF_HUMAN;Replicates CUT&RUN experiments on DE cells using CTCF Millipore 07-729 with a digestion time of 3min;;;4DNEXCQSNNEF;1;1;4DNEXQSRWYGR;yes;;
4DNESN18Y5S3;CUT&RUN;human;Protein:CTCF_HUMAN;Replicates CUT&RUN experiments on DE cells using CTCF Millipore 07-729 with a digestion time of 3min;;;4DNEXKILXU1Q;2;1;4DNEXZPAE6HZ;yes;;
4DNESNHDE8QH;CUT&RUN;human;Protein:Suz12 (Q15022);Replicates CUT&RUN experiments on H1 cells using Suz12 (ab12073) with a digestion time of 3min;;;4DNEXHBQM5M3;1;1;4DNEXN4L7XQZ;yes;;
4DNESNHDE8QH;CUT&RUN;human;Protein:Suz12 (Q15022);Replicates CUT&RUN experiments on H1 cells using Suz12 (ab12073) with a digestion time of 3min;;;4DNEX4WAJESV;2;1;4DNEXCMJGSS5;yes;;
4DNESDRL8DZM;CUT&RUN;human;Protein:Ring2 (Q99496);Replicates CUT&RUN e

4DNESWC8XDNG;CUT&RUN;human;Protein:H3K4me2;Replicates of CUT&RUN on K562 cells using H3K4me2 (Upstate 07-030) with a digestion time of 3min;;;4DNEXERG7TH2;2;1;4DNEX3ATIEZJ;yes;;
4DNESWC8XDNG;CUT&RUN;human;Protein:H3K4me2;Replicates of CUT&RUN on K562 cells using H3K4me2 (Upstate 07-030) with a digestion time of 3min;;;4DNEX1YDKYUO;3;1;4DNEX3ATIEZJ;yes;;
4DNESWC8XDNG;CUT&RUN;human;Protein:H3K4me2;Replicates of CUT&RUN on K562 cells using H3K4me2 (Upstate 07-030) with a digestion time of 3min;;;4DNEXWWYU2FY;4;1;4DNEX3ATIEZJ;yes;;
4DNESAY5QDKX;CUT&RUN;human;Protein:H3K27ac;Replicates of CUT&RUN on K562 cells using H3K27ac (ab4729) with a digestion time of 1min;;;4DNEX6XMZ23B;1;1;4DNEX3ATIEZJ;yes;;
4DNESPUVB1KT;CUT&RUN;human;Protein:CTCF_HUMAN;Replicates of CUT&RUN on K562 cells using CTCF (Millipore 07-729) with a digestion time of 5min;;;4DNEXK4GJVWC;1;1;4DNEX3ATIEZJ;yes;;
4DNESXIPE9SY;CUT&RUN;human;no target;Replicates of CUT&RUN on H1 cells using Rb anti Ms IgG (ab46540) with a digesti

4DNESHFWJJJZ;ChIP-seq;mouse;Protein:Mouse CTCF (Q61164);Replicates of CTCF ChIP-seq on wild-type Patski cells;;;4DNEXB5LHEM5;1;1;4DNEXLQSMP3P;yes;;300-600
4DNESSNWXHXK;ChIP-seq;human;Protein:H3K27ac;Histone modification H3K27ac ChIP-seq;;;4DNEXYQ6TC5B;1;1;4DNEXW3RWUAH;yes;;
4DNESSNWXHXK;ChIP-seq;human;Protein:H3K27ac;Histone modification H3K27ac ChIP-seq;;;4DNEXV57K68O;1;2;4DNEXMOG6F4W;yes;;
4DNESWX32ZCG;ChIP-seq;human;Protein:H3K4me3;Histone modification H3K4me3 ChIP-seq;;;4DNEXYOW7TCB;1;1;4DNEXW3RWUAH;yes;;
4DNESWX32ZCG;ChIP-seq;human;Protein:H3K4me3;Histone modification H3K4me3 ChIP-seq;;;4DNEXBMMVBQW;1;2;4DNEXMOG6F4W;yes;;
4DNES9WNNK52;ChIP-seq;human;Protein:H3K4me1;Histone modification H3K4me1 ChIP-seq;;;4DNEX1UOUWPM;1;1;4DNEXW3RWUAH;yes;;
4DNES9WNNK52;ChIP-seq;human;Protein:H3K4me1;Histone modification H3K4me1 ChIP-seq;;;4DNEXD6Z69AC;1;2;4DNEXMOG6F4W;yes;;


In [19]:
text = '''control
4DNEX7ICLCEK
4DNEXZTMNGRQ
4DNEX7ICLCEK
4DNEXZTMNGRQ
4DNEX7ICLCEK
4DNEXZTMNGRQ
4DNEX7ICLCEK
4DNEXZTMNGRQ
no info
no info
4DNEX9SC3PDL
4DNEXFLHRRKB
4DNEX9SC3PDL
4DNEXFLHRRKB
4DNEX9SC3PDL
4DNEXFLHRRKB
4DNEX3ATIEZJ
4DNEXRKHOMEW
4DNEX6C8NI2O
4DNEX25CHVSY
4DNEXEZFZZVS
4DNEXHCTLOZC
4DNEXEZFZZVS
4DNEXHCTLOZC
4DNEXEZFZZVS
4DNEXHCTLOZC
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
no info
no info
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
no info
no info
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEXFCY2R8H
4DNEXLH32WQP
4DNEX4QCG8RU
4DNEXGX9TY6L
4DNEXIG7INVJ
4DNEX2YTDQGM
4DNEXGX9TY6L
4DNEXIG7INVJ
4DNEX2YTDQGM
4DNEXSJUQGDQ
4DNEXXMOKLHD
4DNEXO9QSVCK
4DNEXT4MG8XY
4DNEXT4MG8XY
no info
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
4DNEX3ATIEZJ
no info
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXT4MG8XY
4DNEXVDUDPJN
no info
no info
4DNEXUEPMOTS
no info
4DNEX4FVGWNU
no info
4DNEXO4ZBUGU
4DNEXVIJQJT1
4DNEXOKS6IC2
no info
no info
no info
4DNEX4OLAN3C
4DNEX993GF46
no info
no info
4DNEX7DTD1A9
4DNEX7DTD1A9
4DNEX7DTD1A9
4DNEXLQSMP3P
4DNEXW3RWUAH
4DNEXMOG6F4W
4DNEXW3RWUAH
4DNEXMOG6F4W
4DNEXW3RWUAH
4DNEXMOG6F4W'''

als = [i for i in text.split('\n') if i.startswith('4DN')]
print(len(als))

128


In [20]:
t2 = '''4DNEXEC5YC8E
4DNEX3W2ZYZO
4DNEXQSRWYGR
4DNEXZPAE6HZ
4DNEXN4L7XQZ
4DNEXCMJGSS5
4DNEX3ATIEZJ
4DNEXT4MG8XY
4DNEXVDUDPJN
4DNEXW3RWUAH
4DNEXUEPMOTS
4DNEX4FVGWNU
4DNEXO4ZBUGU
4DNEXVIJQJT1
4DNEXOKS6IC2
4DNEX4OLAN3C
4DNEX993GF46'''

for t in t2.split('\n'):
    if t in als:
        print('control')

control
control
control
control
control
control
control
control
control
control
control
control
control
control
control
